In [157]:
pip install requests
pip install plotly

SyntaxError: invalid syntax (<ipython-input-157-f805a5b58ba0>, line 1)

In [96]:
import requests
#import pandas as pd
import plotly
import os
import plotly.graph_objects as go

In [20]:
plat="Platinum Imperia Mining Ltd"

In [145]:
class factionLocations:
    def __init__(self,referenceFaction):
        self.faction=referenceFaction  
        self.presentSystems=[]
        #self.listFactionLocations()
    def listFactionLocations(self):
        self.factionLocations=[]
        EDDBfactions="https://elitebgs.app/api/ebgs/v4/factions"
        URL =os.path.join(EDDBfactions)
        PARAMS = {"name":self.faction} 
        r = requests.get(url = URL, params = PARAMS) 
        data = r.json()
        for item in data["docs"]:
            for system in item["faction_presence"]:
                self.factionLocations.append( system["system_name"])
        print("The faction %s is located in the following systems:"%self.faction)
        for item in self.factionLocations:
            print(item)
        #=======================================================================
        # for item in data["docs"]:
        #     for system in item["faction_presence"]:
        #         print system
        #=======================================================================
        return self.factionLocations#, data["docs"]

In [146]:
x=factionLocations(referenceFaction=plat)

In [152]:
class factionInfluence:
    def __init__(self,system,graph=0):
        #self.faction=referenceFaction
        #self.presentSystems=factionLocations(referenceFaction)
        self.influences={}
        self.influences["factions"]=[]
        self.influences["inf"]=[]
        self.system=system
        self.graph=graph
        #self.getInf()

    def getInf(self):
        endpoint="https://www.edsm.net/api-system-v1/factions"
        PARAMS = {"systemName":self.system} 
        r = requests.get(url=endpoint, params = PARAMS) 
        data = r.json()
        self.influences["controlling"]=data["controllingFaction"]["name"]
        #print(data["controllingFaction"]["name"])
        for item in data["factions"]:
            self.influences["factions"].append(item["name"])
            self.influences["inf"].append(item["influence"]*100)
        
        if self.graph:
            fig = go.Figure(
                data=[go.Bar(x=self.influences["factions"],y=self.influences["inf"])],
                layout_title_text="%s Influences"%self.system
            )
            fig.show()
        else:
            for key,value in self.influences.items():
                print(key,value)


In [153]:
ptl=factionLocations(referenceFaction=plat)
systems=ptl.listFactionLocations()

for system in systems:
    plotter=factionInfluence(system=system,graph=1)
    plotter.getInf()

The faction Platinum Imperia Mining Ltd is located in the following systems:
Wu Jinifex
Paesia
Assiones
HIP 76713
Khan Gubii
Hou Yangk
Tehuevisc
Dogoneja
Yamandara
HIP 79315
Pontar
Scorpii Sector JW-W b1-3
Col 285 Sector QE-M b22-3
Awa
Nadur
Orisattu
HIP 77818
Fer Doira
Lahaunai
